In [80]:
using Plots
using Optim
using Random
using StatsBase
using Statistics
using Distributions
using LinearAlgebra

Inicialmente, vamos elaborar funções para gerar jogadores aleatórios e simular campeonatos, além de outra para, dados os resultados, calcular a verossimilhança dos parâmetros em teste (função que deve ser maximizada).

```create_players``` recebe como parâmetros a quantidade de clubes e jogadores em cada clube e cria jogadores aleatórios.

```simulating_game``` recebe dois clubes e uma quantidade de simulações e simula os jogos entre esses dois clubes por meio de uma Poisson com média $\lambda = \frac{\lambda_a}{\lambda_d}$, onde $\lambda_a$ é o somatório das forças dos jogadores do clube atacante ($\mathcal{C}_a$) e $\lambda_d$ é o somatório das forças dos jogadores do clube defensor ($\mathcal{C}_d$).

```create_games``` recebe a quantidade de temporadas, clubes e jogadores por clube e cria jogos entre todos os clubes simulando essas temporadas.

```likelihood``` recebe os resultados e um conjunto de parâmetros e retorna a log verossimilhança negativa desses parâmetros com o resultado.

```gradient``` gradiente da função likelihood.

As duas últimas funções podem ser melhores explicadas matematicamente, dessa forma, como explicado anteriormente, cada placar é composto por duas variáveis aleatórias vindas de uma Poisson com média igual a razão das somas das forças dos dois clubes, assim seja $k$ a quantidade de gols de um clube contra outro, $\lambda_a$ a soma das forças do clube que marcou os gols e $\lambda_d$ do clube que sofreu. Dessa forma, a verossimilhança desse pedaço de placar é dada por $P(X = k | proficiências) = \frac{\lambda^k\exp{-\lambda}}{k!}$, onde $\lambda = \frac{\lambda_a}{\lambda_d}$. Note que a verossimilhança total é o produto de todos os "pedaços" de placar, mas, como cada fator desses é um valor em $[0, 1]$, isso se torna muito pequeno, podendo ocorrer underflow error, por isso opta-se a usar o log da verossimilhança (como o log é uma função estritamente crescente, a comparação entre os resultados é mantida). Por fim, os fatores se tornam parcelas, agora todas negativas, então multiplicamos por $-1$, transformando essa métrica num valor positivo e, mais importante, mudando o problema para um problema de minimização.

Agora, como citado no parágrafo anterior, cada parcela da verossimilhança é dada pela expressão $-\log{\frac{\lambda^k\exp{-\lambda}}{k!}}$, dessa forma, para cada jogador $j$, temos que a derivada dessa parcela, em relação ao jogador $j$, será:
  - $0$, se $j \notin \mathcal{C}_a \land j \notin \mathcal{C}_d$;
  - $\dfrac{\lambda_d - \lambda_a}{\lambda_d^2} + k \left(\dfrac{1}{\lambda_d} - \dfrac{1}{\lambda_a}\right)$, se $j \in \mathcal{C}_a \land j \in \mathcal{C}_d$;
  - $\dfrac{1}{\lambda_d} - \dfrac{k}{\lambda_a}$, se $j \in \mathcal{C}_a \land j \notin \mathcal{C}_d$ e;
  - $-\dfrac{\lambda_a}{\lambda_d^2} + \dfrac{k}{\lambda_d}$, se $j \notin \mathcal{C}_a \land j \in \mathcal{C}_d$.
  
Assim, passando por cada jogo e fazendo as derivadas parciais em relação a cada jogador, chegamos ao gradiente da função de verossimilhança.

In [81]:
function creat_players(clubs, players_per_club, mean = 0, var = 1, lb = 0, ub = 100)
    all_players = rand(Truncated(Normal(mean, var), lb, ub), (2 * clubs * players_per_club, 2))
    all_players[:, 1] = all_players[:, 1] / (sum(all_players[:, 1]) / clubs)
    all_players[:, 2] = all_players[:, 2] / (sum(all_players[:, 2]) / clubs)
    players = Dict{Int64, Vector{Float64}}()
    for i in 1:(clubs * players_per_club)
        players[i] = all_players[i, :];
    end
    return players
end

creat_players(20, 11)

Dict{Int64, Vector{Float64}} with 220 entries:
  56  => [0.0248283, 0.0598957]
  35  => [0.0418642, 0.0313879]
  60  => [0.0617867, 0.0255398]
  220 => [0.107608, 0.0268199]
  67  => [0.00262926, 0.0500701]
  215 => [0.0175409, 0.0797272]
  73  => [0.0291035, 0.0703364]
  115 => [0.146039, 0.0619473]
  112 => [0.124281, 0.105112]
  185 => [0.0669452, 0.0562891]
  86  => [0.029753, 0.0247484]
  168 => [0.0186791, 0.110752]
  207 => [0.0102838, 0.027709]
  183 => [0.00089305, 0.0109503]
  177 => [0.114682, 0.115224]
  12  => [0.0577733, 0.0214049]
  75  => [0.0233652, 0.0622802]
  23  => [0.0426478, 0.0343346]
  111 => [0.0699207, 0.121672]
  41  => [0.0642761, 0.058417]
  68  => [0.0645152, 0.0420932]
  82  => [0.0428453, 0.0780191]
  130 => [0.00745266, 0.0718609]
  125 => [0.0184421, 0.0916608]
  77  => [0.0456007, 0.10198]
  ⋮   => ⋮

In [82]:
# function creat_players(clubs, players_per_club, mean = 0, var = 1, lb = 0, ub = 100)
#     all_players_atk = rand(Truncated(Normal(mean, var), lb, ub), clubs * players_per_club);
#     all_players_atk = all_players_atk / (sum(all_players_atk) / clubs);
#     players_atk = Dict{Int64, Float64}(zip(1:clubs * players_per_club, all_players_atk));
    
#     all_players_def = rand(Truncated(Normal(mean, var), lb, ub), clubs * players_per_club);
#     all_players_def = all_players_def / (sum(all_players_atk) / clubs);
#     players_def = Dict{Int64, Float64}(zip(1:clubs * players_per_club, all_players_def));
#     return players_atk, players_def
# end;

# atk, def = creat_players(20, 11);
# atk, def

In [83]:
function simulating_game(λatk₁, λatk₂, λdef₁, λdef₂, sims = 1000000)
    X₁ = Poisson(λatk₁ / λdef₂);
    X₂ = Poisson(λatk₂ / λdef₁);
    Y₁ = rand(X₁, sims);
    Y₂ = rand(X₂, sims);
    wins₁ = sum(Y₁ .> Y₂);
    draws = sum(Y₁ .== Y₂);
    wins₂ = sum(Y₁ .< Y₂);
    return wins₁, draws, wins₂
end;

In [84]:
function create_games(seasons, clubs, ppc)
    results = [[[0 for i in 1:4] for j in 1:clubs * (clubs - 1)] for s in 1:seasons];
    players = creat_players(clubs, ppc);
    squads = [];
    for s in 1:seasons
        line = 1;
        append!(squads, [reshape(shuffle(collect(1:length(players))), (clubs, ppc))]);
        clubs_atks = convert(Matrix{Float64}, (deepcopy(last(squads))));
        clubs_defs = convert(Matrix{Float64}, (deepcopy(last(squads))));
        for i in 1:clubs
            for j in 1:ppc
                clubs_atks[i, j] = players[clubs_atks[i, j]][1];
                clubs_defs[i, j] = players[clubs_defs[i, j]][2];
            end
        end

        for j in 1:clubs
            for k in 1:clubs
                if j != k
                    Xⱼ = Poisson(sum(clubs_atks[j, :]) / sum(clubs_defs[k, :]));
                    Xₖ = Poisson(sum(clubs_atks[k, :]) / sum(clubs_defs[j, :]));
                    results[s][line][1] = j;
                    results[s][line][2] = rand(Xⱼ);
                    results[s][line][3] = rand(Xₖ);
                    results[s][line][4] = k;
                    line += 1;
                end
            end
        end
    end
    return results, squads, players
end;

results, squads, players = create_games(10, 20, 11);

In [85]:
function likelihood(players, results, squads)
    if typeof(players) != Dict{Int64, Vector{Float64}}
        all_players = Dict{Int64, Vector{Float64}}()
        if size(players, 2) != 2
            players = reshape(players, (Int(length(players) // 2), 2));
        end
        
        for i in 1:size(players, 1)
            all_players[i] = players[i, :];
        end
    else
        all_players = players;
    end
    loglikelihood = 0
    for i in 1:length(squads)
        clubs, ppc = size(squads[i]);
        clubs_atks = convert(Matrix{Float64}, (deepcopy(squads[i])));
        clubs_defs = convert(Matrix{Float64}, (deepcopy(squads[i])));
        for j in 1:size(squads[i], 1)
            for k in 1:size(squads[i], 2)
                clubs_atks[j, k] = all_players[squads[i][j, k]][1];
                clubs_defs[j, k] = all_players[squads[i][j, k]][2];
            end
        end
        for j in 1:length(results[i])
            clubₕ, scoreₕ, scoreₐ, clubₐ = results[i][j]
            loglikelihood -= logpdf(Poisson(sum(clubs_atks[clubₕ, :]) / sum(clubs_defs[clubₐ, :])),
                                    scoreₕ)
            loglikelihood -= logpdf(Poisson(sum(clubs_atks[clubₐ, :]) / sum(clubs_defs[clubₕ, :])),
                                    scoreₐ)
        end
    end
    
    return loglikelihood
end

likelihood (generic function with 1 method)

In [86]:
results, squads, players = create_games(10, 20, 11);
println(likelihood(players, results, squads));
players = rand((1, 2), (2 * 20 * 11, ));
println(likelihood(players, results, squads));

10070.610480049134
10648.134451724087


In [87]:
# FALTA EDITAR ESSA
function gradient(players, results, squads)
    if typeof(players) != Dict{Int64, Vector{Float64}}
        all_players = Dict{Int64, Vector{Float64}}()
        if size(players, 2) != 2
            players = reshape(players, (Int(length(players) // 2), 2));
        end
        
        for i in 1:size(players, 1)
            all_players[i] = players[i, :];
        end
    else
        all_players = players;
    end
    p = size(players, 1)
    ∇likelihood = zeros(2 * length(all_players))
    for season in 1:length(results)
        for game in 1:length(results[season])
            clubₕ, kₕ, kₐ, clubₐ = results[season][game]
            λ₁ₕ, λ₁ₐ, λ₂ₕ, λ₂ₐ = 0, 0, 0, 0
            Cₕ, Cₐ = squads[season][clubₕ, :], squads[season][clubₐ, :]
            for k in 1:size(squads[season], 2)
                λ₁ₕ += all_players[Cₕ[k]][1]
                λ₁ₐ += all_players[Cₐ[k]][1]
                λ₂ₕ += all_players[Cₕ[k]][2]
                λ₂ₐ += all_players[Cₐ[k]][2]
            end
            
            for i in 1:p
                j = i + p
                ∇likelihood[i] -= (kₕ/λ₁ₕ - 1 / λ₂ₐ) * (i in Cₕ)
                ∇likelihood[i] -= (kₐ/λ₁ₐ - 1 / λ₂ₕ) * (i in Cₐ)
                ∇likelihood[j] -= (- kₕ/λ₂ₐ + λ₁ₕ / (λ₂ₐ^2)) * (i in Cₐ)
                ∇likelihood[j] -= (- kₐ/λ₂ₕ + λ₁ₐ / (λ₂ₕ^2)) * (i in Cₕ)
            end
        end
    end
    
    return ∇likelihood
end;

In [88]:
gradient(players, results, squads)

440-element Vector{Float64}:
 -1.980643255952856
 -0.6302310735360279
 -1.6449110143382584
 -4.730234144940033
 -0.9803176871288337
  0.27060373407122706
  1.2067454174652326
 -1.6219241542770948
  2.801218260583584
  0.1048529600774169
  0.7116751626426535
 -0.11894857808170559
 -0.8875250437556936
  ⋮
  3.8210181468053444
 -0.1678368229099921
 -0.645351802703803
  0.825755026263517
  1.6657489245426607
  0.6779309940229663
  0.192604099257733
  3.2908557713705
  2.448676146384482
 -2.835985869838135
 -2.263712229990038
  5.112927463406407

# Otimizando

In [89]:
seasons = 10
n_clubs = 20
ppc = 11

# compilando
results, squads, players = @time create_games(seasons, n_clubs, ppc)
clubs = convert(Matrix{Float64}, (deepcopy(last(squads))))
for i in 1:n_clubs
    for j in 1:ppc
        clubs[i, j] = players[clubs[i, j]]
    end
end

@time likelihood(players, results, squads)
@time gradient(players, results, squads, zeros(length(players)))

# reexecutando
results, squads, players = @time create_games(seasons, n_clubs, ppc)
clubs = convert(Matrix{Float64}, (deepcopy(last(squads))))
for i in 1:n_clubs
    for j in 1:ppc
        clubs[i, j] = players[clubs[i, j]]
    end
end

@time likelihood(players, results, squads)
@time gradient(players, results, squads, zeros(length(players)))
println()

  0.024141 seconds (73.79 k allocations: 4.015 MiB)


LoadError: MethodError: [0mCannot `convert` an object of type [92mVector{Float64}[39m[0m to an object of type [91mFloat64[39m
[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::Base.TwicePrecision[39m) where T<:Number at twiceprecision.jl:250
[0m  convert(::Type{T}, [91m::AbstractChar[39m) where T<:Number at char.jl:180
[0m  convert(::Type{T}, [91m::CartesianIndex{1}[39m) where T<:Number at multidimensional.jl:136
[0m  ...

In [90]:
values(players)

ValueIterator for a Dict{Int64, Vector{Float64}} with 220 entries. Values:
  [0.023575951040244635, 0.06583108069906998]
  [0.0009186737867302627, 0.037687501143288504]
  [0.02001256464226446, 0.06059764311025076]
  [0.01139100378654155, 0.012073373478108359]
  [0.030756989125094683, 0.023176187806510914]
  [0.024836261901395058, 0.03187689603308066]
  [0.011872905277594105, 0.0296950342472347]
  [0.04931278029475757, 0.024276698559764578]
  [0.019589657695884385, 0.09757786361406724]
  [0.013525493242673384, 0.06387452329271752]
  [0.06947450599777226, 0.026358347815781887]
  [0.030181591867424504, 0.052367232769549917]
  [0.02175542176273661, 0.05738477543635949]
  [0.0771154131332652, 0.037540438511127025]
  [0.025283949454591552, 0.02372421391084237]
  [0.029401325125274474, 0.10330888959644262]
  [0.00910333896418747, 0.018307012422791966]
  [0.0500369739636586, 0.02004158846491057]
  [0.027638233392130264, 0.034095901126158304]
  [0.02581500268278313, 0.06170698414811822]
  [0.01

In [5]:
a = zeros(220)
opt_a = []
opt_value = Inf
for i in 1:220
    a = rand(220)
#     for j in 1:220
#         a[j] = 1 / rand(1:10)
#         println(a)
#     end
    value = likelihood(a, results, squads)
    if opt_value > value
        opt_a = a
        opt_value = value
    end
end

opt_value, likelihood(players, results, squads)

(10560.228268364872, 10005.691447429792)

# Teste para Otimização

In [6]:
f(x) = likelihood(x, results, squads)
g(∇likelihood, x) = gradient(x, results, squads, ∇likelihood)
lower = zeros(220)
upper = 20 * ones(220)
x_inicial = rand(220)
od = OnceDifferentiable(f, g, x_inicial)
@time res1  = optimize(od,
                       lower,
                       upper,
                       x_inicial,
                       Fminbox(GradientDescent()),
                       Optim.Options(iterations = 1000))
@time res2  = optimize(od,
                       lower,
                       upper,
                       x_inicial,
                       Fminbox(NelderMead()),
                       Optim.Options(iterations = 1000))
@time res3 = optimize(x -> likelihood(x, results, squads),
                      lower,
                      upper,
                      x_inicial,
                      Fminbox(NelderMead()),
                      Optim.Options(iterations = 1000))

println()

  5.482042 seconds (21.72 M allocations: 1.471 GiB, 8.51% gc time, 57.82% compilation time)
  5.969408 seconds (38.98 M allocations: 1.708 GiB, 3.93% gc time, 22.05% compilation time)
2735.770718 seconds (18.61 G allocations: 803.971 GiB, 4.13% gc time, 0.02% compilation time)



In [7]:
Optim.converged(res1), Optim.minimizer(res1), Optim.minimum(res1), likelihood(players, results, squads)

(true, [0.8064053988114606, 0.7083505250259312, 0.38896408674999017, 0.465292671883069, 0.21192459102565175, 0.05524560321118832, 0.03523755170947118, 0.6273807703084375, 0.1274308824242314, 0.8584145403039296  …  0.9076812210334986, 0.3917834086618406, 0.22962074812279298, 0.5259030236350959, 0.8970130429558458, 0.608138140231612, 0.25857237624124796, 0.6419585750004564, 0.04151790216435258, 0.8405438188487186], 10739.503593434052, 10005.691447429792)

In [8]:
max_lik_players = Dict{Int64, Float64}(zip(1:length(Optim.minimizer(res1)), Optim.minimizer(res1)))

original_players = zeros(220)
optimized_players = zeros(220)

for i in 1:220
    original_players[i] = players[i]
    optimized_players[i] = max_lik_players[i]
end

original_players /= original_players[1]
optimized_players /= optimized_players[1]
grad = gradient(Optim.minimizer(res1), results, squads, zeros(length(players)))
println("Correlação de Pearson: ", cor(original_players, optimized_players))
println("Correlação de Spearman: ", corspearman(original_players, optimized_players))
println("Correlação de Kendall: ", corkendall(original_players, optimized_players))
println("||Gradiente||: ", norm(grad))

Correlação de Pearson: 0.006183945148015716
Correlação de Spearman: -0.003596242597048399
Correlação de Kendall: -0.0021585720215857203
||Gradiente||: 329.68311552961467


In [9]:
Optim.converged(res2), Optim.minimizer(res2), Optim.minimum(res2), likelihood(players, results, squads)

(true, [0.8064053988114606, 0.7083505250259312, 0.38896408674999017, 0.465292671883069, 0.21192459102565175, 0.05524560321118832, 0.03523755170947118, 0.6273807703084375, 0.1274308824242314, 0.8584145403039296  …  0.9076812210334986, 0.3917834086618406, 0.22962074812279298, 0.5259030236350959, 0.8970130429558458, 0.608138140231612, 0.25857237624124796, 0.6419585750004564, 0.04151790216435258, 0.8405438188487186], 10719.28862408449, 10005.691447429792)

In [10]:
max_lik_players = Dict{Int64, Float64}(zip(1:length(Optim.minimizer(res2)), Optim.minimizer(res2)))

original_players = zeros(220)
optimized_players = zeros(220)

for i in 1:220
    original_players[i] = players[i]
    optimized_players[i] = max_lik_players[i]
end

original_players /= original_players[1]
optimized_players /= optimized_players[1]
grad = gradient(Optim.minimizer(res2), results, squads, zeros(length(players)))
println("Correlação de Pearson: ", cor(original_players, optimized_players))
println("Correlação de Spearman: ", corspearman(original_players, optimized_players))
println("Correlação de Kendall: ", corkendall(original_players, optimized_players))
println("||Gradiente||: ", norm(grad))

Correlação de Pearson: 0.01722695267248364
Correlação de Spearman: 0.003408034350822425
Correlação de Kendall: 0.0026567040265670404
||Gradiente||: 320.7019309829996


In [ ]:
Optim.converged(res3), Optim.minimizer(res3), Optim.minimum(res3), likelihood(players, results, squads)

In [92]:
max_lik_players = Dict{Int64, Float64}(zip(1:length(Optim.minimizer(res3)), Optim.minimizer(res3)))

original_players = zeros(220)
optimized_players = zeros(220)

for i in 1:220
    original_players[i] = players[i]
    optimized_players[i] = max_lik_players[i]
end

original_players /= original_players[1]
optimized_players /= optimized_players[1]
grad = gradient(Optim.minimizer(res3), results, squads, zeros(length(players)))
println("Correlação de Pearson: ", cor(original_players, optimized_players))
println("Correlação de Spearman: ", corspearman(original_players, optimized_players))
println("Correlação de Kendall: ", corkendall(original_players, optimized_players))
println("||Gradiente||: ", norm(grad))

LoadError: UndefVarError: res3 not defined

In [91]:
@time res4  = optimize(od,
                       lower,
                       upper,
                       Optim.minimizer(res3),
                       Fminbox(GradientDescent()),
                       Optim.Options(iterations = 1000))

res4

LoadError: UndefVarError: res3 not defined

In [14]:
Optim.minimizer(res3) == Optim.minimizer(res4)

true